In [11]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import tldextract
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize tldextract for domain extraction
extractor = tldextract.TLDExtract(cache_dir=False)

def analyze_url(url):
    features = {}
    
    # Parse URL components
    extracted = extractor(url)
    protocol = urlparse(url).scheme  # Get the protocol (http or https)
    print(protocol)
    full_domain = protocol + ('.' + extracted.subdomain if extracted.subdomain else '') + '.' + extracted.domain
    
    # 1. Domain Length (excluding protocol)
    features['DomainLength'] = len(('.' + extracted.subdomain if extracted.subdomain else '') + '.' + extracted.domain)
    
    # 2. Is Domain an IP Address?
    features['IsDomainIP'] = 1 if re.match(r'^\d{1,3}(\.\d{1,3}){3}$', extracted.domain) else 0
    
    # 3. URL Similarity Index (Cosine similarity with risky keywords)
    features['URLSimilarityIndex'] = calculate_similarity_index(url)
    
    # 4. Char Continuation Rate (rate of repeated characters in URL)
    features['CharContinuationRate'] = calculate_continuation_rate(url)
    
    # 5. TLD Legitimate Probability (this could be complex, so it's a placeholder)
    features['TLDLegitimateProb'] = 0.9  # Placeholder example value
    
    # 6. URL Character Probability (basic probability check using character frequencies)
    features['URLCharProb'] = calculate_char_probability(url)
    
    # 7. TLD Length (length of the TLD part)
    features['TLDLength'] = len(extracted.suffix)
    
    # 8. Number of Subdomains
    features['NoOfSubDomain'] = len(extracted.subdomain.split('.')) if extracted.subdomain else 0
    
    # 9. Obfuscation: Check if URL has obfuscation patterns (%20, _, -, etc.)
    features['HasObfuscation'] = 1 if has_obfuscation(url) else 0
    features['NoOfObfuscatedChar'] = len(re.findall(r'%|\.|_|-|@|%20', url))
    features['ObfuscationRatio'] = features['NoOfObfuscatedChar'] / len(url) if len(url) > 0 else 0
    
    # 10. Letters in URL (only letters)
    features['NoOfLettersInURL'] = sum(c.isalpha() for c in url)
    features['LetterRatioInURL'] = features['NoOfLettersInURL'] / len(url) if len(url) > 0 else 0
    
    # 11. Digits in URL (only digits)
    features['NoOfDegitsInURL'] = sum(c.isdigit() for c in url)
    features['DegitRatioInURL'] = features['NoOfDegitsInURL'] / len(url) if len(url) > 0 else 0
    
    # 12. Count specific symbols in URL
    features['NoOfEqualsInURL'] = url.count('=')
    features['NoOfQMarkInURL'] = url.count('?')
    features['NoOfAmpersandInURL'] = url.count('&')
    features['NoOfOtherSpecialCharsInURL'] = len(re.findall(r'[^a-zA-Z0-9:/?&=]', url))
    features['SpacialCharRatioInURL'] = features['NoOfOtherSpecialCharsInURL'] / len(url) if len(url) > 0 else 0
    
    # 13. HTTPS check
    features['IsHTTPS'] = 1 if url.startswith('https://') else 0
    
    # Webpage-specific features (scraping)
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        features['LineOfCode'] = len(soup.prettify().splitlines())
        features['LargestLineLength'] = max(len(line) for line in soup.prettify().splitlines())
        
        title_tag = soup.find('title')
        features['HasTitle'] = 1 if title_tag else 0
        features['DomainTitleMatchScore'] = calculate_domain_title_match_score(title_tag.text if title_tag else '', extracted.domain)
        url_path = urlparse(url).path
        features['URLTitleMatchScore'] = calculate_url_title_match_score(title_tag.text if title_tag else '', url_path)
        
        features['HasFavicon'] = 1 if soup.find('link', rel='icon') else 0
        features['Robots'] = 1 if requests.get(url + '/robots.txt').status_code == 200 else 0
        features['IsResponsive'] = 1 if soup.find('meta', attrs={'name': 'viewport'}) else 0
        features['NoOfURLRedirect'] = len(response.history)
        features['NoOfSelfRedirect'] = len(soup.find_all('a', href=lambda x: x and x.startswith(url)))
        features['HasDescription'] = 1 if soup.find('meta', attrs={'name': 'description'}) else 0
        features['NoOfPopup'] = count_popups(soup)
        features['NoOfiFrame'] = len(soup.find_all('iframe'))
        features['HasExternalFormSubmit'] = 1 if soup.find('form', action=lambda x: x and not x.startswith('/')) else 0
        features['HasSocialNet'] = 1 if soup.find('a', href=lambda x: 'facebook.com' in x or 'twitter.com' in x) else 0
        features['HasSubmitButton'] = 1 if soup.find('input', type='submit') else 0
        features['HasHiddenFields'] = 1 if soup.find('input', type='hidden') else 0
        features['HasPasswordField'] = 1 if soup.find('input', type='password') else 0
        features['Bank'] = 1 if 'bank' in url.lower() else 0
        features['Pay'] = 1 if 'pay' in url.lower() else 0
        features['Crypto'] = 1 if 'crypto' in url.lower() else 0
        features['HasCopyrightInfo'] = 1 if soup.find(text=re.compile(r'copyright', re.I)) else 0
        features['NoOfImage'] = len(soup.find_all('img'))
        features['NoOfCSS'] = len(soup.find_all('link', rel='stylesheet'))
        features['NoOfJS'] = len(soup.find_all('script'))
        features['NoOfSelfRef'] = len(soup.find_all('a', href=lambda x: x and x.startswith(url)))
        features['NoOfEmptyRef'] = len(soup.find_all('a', href=''))
        features['NoOfExternalRef'] = len(soup.find_all('a', href=lambda x: x and not x.startswith(url) and not x.startswith('/')))
        
    except Exception as e:
        print(f"Error fetching {url}: {e}")
    
    return pd.DataFrame([features])

# Helper functions
def calculate_continuation_rate(url):
    continuation_count = sum(url[i] == url[i + 1] for i in range(len(url) - 1))
    return continuation_count / (len(url) - 1) if len(url) > 1 else 0

def calculate_similarity_index(url, keywords=["bank", "login", "secure", "account"]):
    vectorizer = TfidfVectorizer().fit_transform([url] + keywords)
    vectors = vectorizer.toarray()
    similarity_matrix = cosine_similarity(vectors)
    return max(similarity_matrix[0][1:])

def calculate_char_probability(url):
    char_counts = {char: url.count(char) for char in set(url)}
    total_chars = len(url)
    probabilities = {char: count / total_chars for char, count in char_counts.items()}
    return sum(probabilities.values()) / len(probabilities) if probabilities else 0

def has_obfuscation(url):
    return bool(re.search(r'%[0-9A-F]{2}|_|-|@|%20', url))

def calculate_domain_title_match_score(title, domain):
    title_words = set(title.lower().split())
    domain_words = set(domain.lower().split('.'))
    return len(title_words & domain_words) / len(title_words) if title_words else 0

def calculate_url_title_match_score(title, path):
    path_words = set(path.lower().split('/'))
    title_words = set(title.lower().split())
    return len(path_words & title_words) / len(path_words) if path_words else 0

def count_popups(soup):
    # Custom function to count possible popups
    return len(soup.find_all('script', src=lambda x: x and 'popup' in x))

# Test with URL
url1 = str(input("Enter a URL to analyze: "))

features_df = analyze_url(url1)
features_df


https


C:\Users\cl501_27\AppData\Local\Temp\ipykernel_11464\1920240273.py:99: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  features['HasCopyrightInfo'] = 1 if soup.find(text=re.compile(r'copyright', re.I)) else 0


,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,NoOfObfuscatedChar,...,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef
0,14,0,0.0,0.173913,0.9,0.058824,2,1,1,3,...,0,0,0,0,2,18,76,0,0,2


In [12]:
features_df.iloc[0]

DomainLength                     14.000000
IsDomainIP                        0.000000
URLSimilarityIndex                0.000000
CharContinuationRate              0.173913
TLDLegitimateProb                 0.900000
URLCharProb                       0.058824
TLDLength                         2.000000
NoOfSubDomain                     1.000000
HasObfuscation                    1.000000
NoOfObfuscatedChar                3.000000
ObfuscationRatio                  0.125000
NoOfLettersInURL                 18.000000
LetterRatioInURL                  0.750000
NoOfDegitsInURL                   0.000000
DegitRatioInURL                   0.000000
NoOfEqualsInURL                   0.000000
NoOfQMarkInURL                    0.000000
NoOfAmpersandInURL                0.000000
NoOfOtherSpecialCharsInURL        3.000000
SpacialCharRatioInURL             0.125000
IsHTTPS                           1.000000
LineOfCode                     1441.000000
LargestLineLength             56689.000000
HasTitle   

In [12]:
a="hdfcbank"
'bank' in a

True